In [ ]:
# pyright:basic
# NOTE: When you check this the next morning, set this up in a jupyter notebook so you can actually see shit.

import ssl

from ee.imagecollection import ImageCollection

ssl._create_default_https_context = ssl._create_stdlib_context

import ee
import geemap.core as geemap

ee.Authenticate()
ee.Initialize(project="THE-PROJECT-ID")

test = ImageCollection("FAO/GHG/1/DROSA_A")

dataset_names = {
    "Temp&Rain": "ECMWF/ERA5/DAILY",
    "DrainedSoil": "FAO/GHG/1/DROSA_A",
    "NDVI": "MODIS/MOD09GA_006_NDVI",
    "Crop&GlobalSupport": "USGS/GFSAD1000_V1",
}


def request_data_for_longitude_lattitude(long, lat, era5, drosa_a, ndvi, gfsad):
    """
    Get the data for the area around long and lat, for each of the dataset, and return it as a tuple.
    """


def get_most_recent_value_at_point(
    dataset_id: str,
    longitude: float,
    latitude: float,
    scale: int = 10,
):
    # 1. Define a point geometry for the location of interest.
    point = ee.Geometry.Point([longitude, latitude])

    # 2. Get the ImageCollection, filter it by the point, and sort by date.
    collection = (
        ee.ImageCollection(dataset_id)
        .filterBounds(point)
        .sort(
            "system:time_start", False
        )  # Sort descending to get the most recent first
    )

    # 3. Get the most recent image.
    most_recent_image = collection.first()

    return most_recent_image


def retrieve_ee_data(dataset_name) -> ImageCollection:
    """
    Dataset is one of:
        Temp&Rainkk
        DrainedSoil
        NDVI
        Crop&GlobalSupport
    """

    return ImageCollection(dataset_names[dataset_name])

In [22]:
# Example script to load and visualize ERA5 climate reanalysis parameters in
# Google Earth Engine

# Daily mean 2m air temperature
era5_2mt = (
    ee.ImageCollection('ECMWF/ERA5/DAILY')
    .select('mean_2m_air_temperature')
    .filter(ee.Filter.date('2019-07-01', '2019-07-31'))
)
display(era5_2mt)

# Daily total precipitation sums
era5_tp = (
    ee.ImageCollection('ECMWF/ERA5/DAILY')
    .select('total_precipitation')
    .filter(ee.Filter.date('2019-07-01', '2019-07-31'))
)

# Daily mean 2m dewpoint temperature
era5_2d = (
    ee.ImageCollection('ECMWF/ERA5/DAILY')
    .select('dewpoint_2m_temperature')
    .filter(ee.Filter.date('2019-07-01', '2019-07-31'))
)

# Daily mean sea-level pressure
era5_mslp = (
    ee.ImageCollection('ECMWF/ERA5/DAILY')
    .select('mean_sea_level_pressure')
    .filter(ee.Filter.date('2019-07-01', '2019-07-31'))
)

# Daily mean surface pressure
era5_sp = (
    ee.ImageCollection('ECMWF/ERA5/DAILY')
    .select('surface_pressure')
    .filter(ee.Filter.date('2019-07-01', '2019-07-31'))
)

# Daily mean 10m u-component of wind
era5_u_wind_10m = (
    ee.ImageCollection('ECMWF/ERA5/DAILY')
    .select('u_component_of_wind_10m')
    .filter(ee.Filter.date('2019-07-01', '2019-07-31'))
)

# Convert pressure levels from Pa to hPa - Example for surface pressure
era5_sp = era5_sp.map(
    lambda image: image.divide(100).set(
        'system:time_start', image.get('system:time_start')
    )
)

# Visualization palette for total precipitation
vis_tp = {
    'min': 0.0,
    'max': 0.1,
    'palette': ['ffffff', '00ffff', '0080ff', 'da00ff', 'ffa400', 'ff0000'],
}

# Visualization palette for temperature (mean, min and max) and 2m dewpoint
# temperature
vis_2mt = {
    'min': 250,
    'max': 320,
    'palette': [
        '000080',
        '0000d9',
        '4000ff',
        '8000ff',
        '0080ff',
        '00ffff',
        '00ff80',
        '80ff00',
        'daff00',
        'ffff00',
        'fff500',
        'ffda00',
        'ffb000',
        'ffa400',
        'ff4f00',
        'ff2500',
        'ff0a00',
        'ff00ff',
    ],
}

# Visualization palette for u- and v-component of 10m wind
vis_wind = {
    'min': 0,
    'max': 30,
    'palette': [
        'ffffff',
        'ffff71',
        'deff00',
        '9eff00',
        '77b038',
        '007e55',
        '005f51',
        '004b51',
        '013a7b',
        '023aad',
    ],
}

# Visualization palette for pressure (surface pressure, mean sea level
# pressure) - adjust min and max values for mslp to 'min':990 and 'max':1050
vis_pressure = {
    'min': 500,
    'max': 1150,
    'palette': [
        '01ffff',
        '058bff',
        '0600ff',
        'df00ff',
        'ff00ff',
        'ff8c00',
        'ff8c00',
    ],
}


# Add layer to map
m = geemap.Map()
m.add_layer(
    era5_tp.filter(ee.Filter.date('2019-07-15')),
    vis_tp,
    'Daily total precipitation sums',
)
m.add_layer(
    era5_2d.filter(ee.Filter.date('2019-07-15')),
    vis_2mt,
    'Daily mean 2m dewpoint temperature',
)
m.add_layer(
    era5_2mt.filter(ee.Filter.date('2019-07-15')),
    vis_2mt,
    'Daily mean 2m air temperature',
)
m.add_layer(
    era5_u_wind_10m.filter(ee.Filter.date('2019-07-15')),
    vis_wind,
    'Daily mean 10m u-component of wind',
)
m.add_layer(
    era5_sp.filter(ee.Filter.date('2019-07-15')),
    vis_pressure,
    'Daily mean surface pressure',
)

m.set_center(21.2, 22.2, 2)
m

Map(center=[22.2, 21.2], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', …

In [23]:
def get_most_recent_value(collection, lon, lat, band_name):
    """
    Get the most recent available value from an ImageCollection at a given lon/lat.
    
    Args:
        collection (ee.ImageCollection): Sorted image collection (newest first).
        lon (float): Longitude.
        lat (float): Latitude.
        band_name (str): Band to extract values from.
    
    Returns:
        float or None: The most recent value found, or None if no valid data exists.
    """
    point = ee.Geometry.Point([lon, lat])
    
    # Get the list of images (server-side to client-side iteration is tricky).
    # Safer: iterate server-side until we find a valid value.
    def iterate_fn(img, prev):
        prev = ee.Dictionary(prev)
        found = ee.Algorithms.If(
            prev.get('found'),
            prev,  # already found, skip
            ee.Algorithms.If(
                img.reduceRegion(
                    reducer=ee.Reducer.first(),
                    geometry=point,
                    scale=1000, # adjust scale to match dataset
                    bestEffort=True
                ).get(band_name),
                ee.Dictionary({'found': True, 'value': img.reduceRegion(
                    reducer=ee.Reducer.first(),
                    geometry=point,
                    scale=1000,
                    bestEffort=True
                ).get(band_name)}),
                prev
            )
        )
        return found
    
    result = collection.iterate(iterate_fn, ee.Dictionary({'found': False}))
    result = ee.Dictionary(result)
    
    # Bring it to client
    if result.get('found').getInfo():
        return result.get('value').getInfo()
    else:
        return None


# Example usage:
dataset = ee.ImageCollection("ECMWF/ERA5/DAILY").sort("system:time_start", False)

lon, lat = 174.7633, -36.8485  # Auckland
value = get_most_recent_value(dataset, lon, lat, "total_precipitation")

print("Most recent value:", value)

Most recent value: 0.00011048652231693268
